In [211]:
# initialisation
import pandas as pd
import numpy as np
import plotly.express as px
import itertools
import joblib
from scipy.stats import *
from scipy.optimize import *
from pandasgui import show
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [212]:
# load the data
raw_data = pd.read_csv('../data/Games Database - Main Page.csv')
#show(raw_data)
#drop fields we won't be needing
raw_data.drop(['Me - Review', 'Percentile', 'Launch Platform(s)', 'Hours Played', 'Completion', 'Year(s) revisited/ continued/ earlier dabbled'], axis = 1, inplace = True)
raw_data

,Game,Rating,Reviewscore,Played on,Launch Year,Play Year,DLC/ Major update played,Publisher,Developer,Franchise,Key words,Remaster Reviewscore,Remaster Release Year,Developer2
0,The Legend of Zelda: Breath of the Wild,100,97,Switch,2017,2017,Yes,Nintendo,Nintendo,The Legend of Zelda,"Adventure, open-world, sandbox, exploration, p...",NaN,NaN,NaN
1,The Legend of Zelda: Tears of the Kingdom,100,96,Switch,2023,2023,No,Nintendo,Nintendo,The Legend of Zelda,"Adventure, open-world, sandbox, exploration, p...",NaN,NaN,NaN
2,God of War (2018),99,94,PS4,2018,2018,No,Sony,Santa Monica Studio,God of War,"Hack and slash, third-person, story-driven",NaN,NaN,NaN
3,Metroid Prime,99,97,Switch,2002,2023,No,Nintendo,Retro Studios,Metroid,"Metroidvania, first-person, shooter, platformi...",94.0,2023.0,NaN
4,The Last of Us,99,95,PS4,2013,2014,Yes,Sony,Naughty Dog,The Last of Us,"Horror, third-person, stealth, survival, story...",94.0,2014.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,CrossfireX,40,42,XBS,2022,2023,Yes,Indie,Smilegate,Crossfire,"First-person, shooter, multiplayer, action",NaN,NaN,Remedy
439,Diablo Immortal,39,65,Mobile,2022,2022,No,Activision-Blizzard,Blizzard,Diablo,"Isometric, hack and slash, action-RPG, looter",NaN,NaN,NaN
440,Fifa 14 Legacy Edition,37,38,Vita,2013,2013,No,EA,EA Sports,Fifa,Sports (football),NaN,NaN,NaN
441,Sunlight,32,78,PC,2021,2023,No,Indie,Krillbite Studio,Sunlight,"Adventure, first-person",NaN,NaN,NaN


In [213]:
# a bit of basic plotting
fig = px.scatter(raw_data, x="Reviewscore", y="Rating", color="Played on", hover_name="Game")
fig

In [214]:
# target to beat - relationship between review score and rating
print('Pearson R - ', pearsonr(raw_data['Reviewscore'],raw_data['Rating']))

Pearson R -  PearsonRResult(statistic=0.7387332054549182, pvalue=1.4080224400319495e-77)


In [215]:
#create set of unique key words from the data frame
str_data = raw_data.copy()
str_data['Key words'] = str_data['Key words'].apply(lambda x: x.split(',')) #convert each entry in key words to a list
lst = str_data['Key words'].to_list() #append all lists together
combine = sum(lst, []) #roll the lists into one
remove_space = [s.strip(' ') for s in combine] #remove leading and trailing spaces in list items
capitalize = [s.upper() for s in remove_space] #convert all characters to upper case
kwords = set(capitalize) #remove duplicate list items
kwords = list(kwords)

# add new columns which track the keywords
kw_data = raw_data.copy()
kw_data['Key words'] = kw_data['Key words'].str.upper()
for i in kwords:
    kw_data[i] = kw_data['Key words'].apply(lambda x: 1 if str(i) in x else 0)

In [216]:
#create set of unique platforms from the data frame
str_data = raw_data.copy()
str_data['Played on'] = str_data['Played on'].apply(lambda x: x.split(',')) #convert each entry in key words to a list
lst = str_data['Played on'].to_list() #append all lists together
combine = sum(lst, []) #roll the lists into one
remove_space = [s.strip(' ') for s in combine] #remove leading and trailing spaces in list items
capitalize_plats = [s.upper() for s in remove_space] #convert all characters to upper case
plats = set(capitalize_plats) #remove duplicate list items
plats = list(plats)

# add new columns which track the platforms
data = kw_data.copy()
data['Played on'] = data['Played on'].str.upper()
for i in plats:
    data[i] = data['Played on'].apply(lambda x: 1 if str(i) in x else 0)

In [217]:
# update publisher to group to other if count less than n
counts_p = data['Publisher'].value_counts() < 3 #creates a series with publisher names as index and values as false, for those not meeting condition
data.loc[data['Publisher'].isin(counts_p.index[counts_p]) == True, 'Publisher'] = 'Other' #sets publisher = other if not in counts
#notes - isin(counts.index[counts]) returns false for all publishers not meeting condition, true for those meeting the condition

#update developer to group to other if count less than n
counts_d = data['Developer'].value_counts() < 2 #creates a series with publisher names as index and values as false, for those not meeting condition
data.loc[data['Developer'].isin(counts_d.index[counts_d]) == True, 'Developer'] = 'Other' #sets publisher = other if not in counts

#update franchise to group to other if count less than n
counts_f = data['Franchise'].value_counts() < 2 #creates a series with publisher names as index and values as false, for those not meeting condition
data.loc[data['Franchise'].isin(counts_f.index[counts_f]) == True, 'Franchise'] = 'Other' #sets publisher = other if not in counts


In [218]:
#only use one of this or the next one
'''
#replace publisher string with its average review score
publisher_avg = (data[['Publisher','Reviewscore']]).groupby(['Publisher']).mean()['Reviewscore']
data = pd.merge(data, publisher_avg, on = 'Publisher')
data.rename(columns= {'Reviewscore_y' : 'Publisher avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)

#replace developer string with its average review score
developer_avg = (data[['Developer','Reviewscore']]).groupby(['Developer']).mean()['Reviewscore']
data = pd.merge(data, developer_avg, on = 'Developer')
data.rename(columns= {'Reviewscore_y' : 'Developer avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)

#replace franchise string with its average review score
franchise_avg = (data[['Franchise','Reviewscore']]).groupby(['Franchise']).mean()['Reviewscore']
data = pd.merge(data, franchise_avg, on = 'Franchise')
data.rename(columns= {'Reviewscore_y' : 'Franchise avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)
'''


"\n#replace publisher string with its average review score\npublisher_avg = (data[['Publisher','Reviewscore']]).groupby(['Publisher']).mean()['Reviewscore']\ndata = pd.merge(data, publisher_avg, on = 'Publisher')\ndata.rename(columns= {'Reviewscore_y' : 'Publisher avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)\n\n#replace developer string with its average review score\ndeveloper_avg = (data[['Developer','Reviewscore']]).groupby(['Developer']).mean()['Reviewscore']\ndata = pd.merge(data, developer_avg, on = 'Developer')\ndata.rename(columns= {'Reviewscore_y' : 'Developer avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)\n\n#replace franchise string with its average review score\nfranchise_avg = (data[['Franchise','Reviewscore']]).groupby(['Franchise']).mean()['Reviewscore']\ndata = pd.merge(data, franchise_avg, on = 'Franchise')\ndata.rename(columns= {'Reviewscore_y' : 'Franchise avg', 'Reviewscore_x' : 'Reviewscore'}, inplace = True)\n"

In [219]:
# one hot encoding of publisher
one_hot = pd.get_dummies(data['Publisher'])
data = data.merge(one_hot, left_index=True, right_index=True)

# one hot encoding of developer
one_hot = pd.get_dummies(data['Developer'])
data = data.merge(one_hot, left_index=True, right_index=True)

# one hot encoding of franchise
one_hot = pd.get_dummies(data['Franchise'])
data = data.merge(one_hot, left_index=True, right_index=True)

In [220]:
# calculate play - launch field
data['Play - launch'] = data['Play Year'] - data['Launch Year']
data['Play - launch'].fillna(0, inplace=True)
# calculate upgraded played field
data.loc[np.isnan(data['Remaster Reviewscore']) == False, 'Remaster played'] = 1
data['Remaster played'].fillna(0, inplace=True)


# calculate latest launch field 
data['Latest launch year'] = data['Remaster Release Year']
data['Latest launch year'].fillna(data['Launch Year'], inplace = True)
# calculate play - latest launch field (remaster)
data['Play - latest launch'] = data['Play Year'] - data['Latest launch year']
data['Play - latest launch'].fillna(0, inplace=True)

# calculate any multiplayer field
data['Key words'] = data['Key words'].str.upper()
data.loc[data['Key words'].str.contains('MULTIPLAYER') == True, 'Any multiplayer'] = 1
data['Any multiplayer'].fillna(0, inplace=True)
# calculate any racing field
data.loc[data['Key words'].str.contains('RACING') == True, 'Any racing'] = 1
data['Any racing'].fillna(0, inplace=True)
# calculate any sports field
data.loc[data['Key words'].str.contains('SPORTS') == True, 'Any sports'] = 1
data['Any sports'].fillna(0, inplace=True)
# calculate any simulation field
data.loc[data['Key words'].str.contains('SIMULATION') == True, 'Any simulation'] = 1
data['Any simulation'].fillna(0, inplace=True)
# calculate any strategy field
data.loc[data['Key words'].str.contains('STRATEGY') == True, 'Any strategy'] = 1
data['Any strategy'].fillna(0, inplace=True)


In [221]:
# convert the yes no fields to 1 or 0
data['DLC/ Major update played'] = data['DLC/ Major update played'].replace({'Yes': 1, 'No': 0})

In [222]:
# tidy missing values
data['Remaster Reviewscore'].fillna(0, inplace=True)

In [223]:
#have another look at the data
data

,Game,Rating,Reviewscore,Played on,Launch Year,Play Year,DLC/ Major update played,Publisher,Developer,Franchise,...,inFAMOUS,Play - launch,Remaster played,Latest launch year,Play - latest launch,Any multiplayer,Any racing,Any sports,Any simulation,Any strategy
0,The Legend of Zelda: Breath of the Wild,100,97,SWITCH,2017,2017,1,Nintendo,Nintendo,The Legend of Zelda,...,0,0,0.0,2017.0,0.0,0.0,0.0,0.0,0.0,0.0
1,The Legend of Zelda: Tears of the Kingdom,100,96,SWITCH,2023,2023,0,Nintendo,Nintendo,The Legend of Zelda,...,0,0,0.0,2023.0,0.0,0.0,0.0,0.0,0.0,0.0
2,God of War (2018),99,94,PS4,2018,2018,0,Sony,Santa Monica Studio,God of War,...,0,0,0.0,2018.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Metroid Prime,99,97,SWITCH,2002,2023,0,Nintendo,Other,Metroid,...,0,21,1.0,2023.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Last of Us,99,95,PS4,2013,2014,1,Sony,Naughty Dog,The Last of Us,...,0,1,1.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,CrossfireX,40,42,XBS,2022,2023,1,Indie,Other,Other,...,0,1,0.0,2022.0,1.0,1.0,0.0,0.0,0.0,0.0
439,Diablo Immortal,39,65,MOBILE,2022,2022,0,Activision-Blizzard,Blizzard,Diablo,...,0,0,0.0,2022.0,0.0,0.0,0.0,0.0,0.0,0.0
440,Fifa 14 Legacy Edition,37,38,VITA,2013,2013,0,EA,Other,Fifa,...,0,0,0.0,2013.0,0.0,0.0,0.0,1.0,0.0,0.0
441,Sunlight,32,78,PC,2021,2023,0,Indie,Other,Other,...,0,2,0.0,2021.0,2.0,0.0,0.0,0.0,0.0,0.0


In [224]:
# prep for model
X = data.copy()
X.drop(['Rating', 'Game', 'Played on', 'Publisher', 'Developer', 'Franchise', 'Key words', 'Remaster Release Year', 'Developer2'], axis = 1, inplace = True)
y = data['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [225]:
#initialize and fit models
model1 = GradientBoostingRegressor(random_state=0)
model1.fit(X_train, y_train)
'''
model2 = LinearRegression()
model2.fit(X_train, y_train)
'''
model3 = RandomForestRegressor(random_state=0)
model3.fit(X_train, y_train)

model4 = AdaBoostRegressor(random_state=0)
model4.fit(X_train, y_train)

model5 = DecisionTreeRegressor(random_state=0)
model5.fit(X_train, y_train)

model6 = BayesianRidge()
model6.fit(X_train, y_train)

model7 = Ridge(random_state=0)
model7.fit(X_train, y_train)

model8 = RidgeCV()
model8.fit(X_train, y_train)

model9 = ElasticNet(random_state=0)
model9.fit(X_train, y_train)

model10 = ElasticNetCV(random_state=0)
model10.fit(X_train, y_train)

model11 = Lasso(random_state=0)
model11.fit(X_train, y_train)

model12 = LassoCV(random_state=0)
model12.fit(X_train, y_train)

model13 = VotingRegressor(estimators=[('Lasso', model12), ('ElasticNet', model10), ('Ridge', model8), ('BayesianRidge', model6)])
model13.fit(X_train, y_train)

VotingRegressor(estimators=[('Lasso', LassoCV(random_state=0)),
                            ('ElasticNet', ElasticNetCV(random_state=0)),
                            ('Ridge', RidgeCV()),
                            ('BayesianRidge', BayesianRidge())])

In [226]:
#output some actual model performance scores
print('Model 1 - Gradient Boosting Regressor')
print('Coefficient of determination - ', model1.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model1.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model1.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model1.predict(X_test)))
print('Std of predictions - ', np.std(model1.predict(X_test)-y_test))
print('')
'''
print('Model 2 - Linear Regressor')
print('Coefficient of determination - ', model2.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model2.predict(X_test)))
'''
print('Model 3 - Random Forest Regressor')
print('Coefficient of determination - ', model3.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model3.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model3.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model3.predict(X_test)))
print('Std of predictions - ', np.std(model3.predict(X_test)-y_test))
print('')

print('Model 4 - AdaBoost Forest Regressor')
print('Coefficient of determination - ', model4.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model4.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model4.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model4.predict(X_test)))
print('Std of predictions - ', np.std(model4.predict(X_test)-y_test))
print('')

print('Model 5 - Decision Tree Regressor')
print('Coefficient of determination - ', model5.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model5.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model5.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model5.predict(X_test)))
print('Std of predictions - ', np.std(model5.predict(X_test)-y_test))
print('')

print('Model 6 - Bayesian Ridge Linear Regressor')
print('Coefficient of determination - ', model6.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model6.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model6.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model6.predict(X_test)))
print('Std of predictions - ', np.std(model6.predict(X_test)-y_test))
print('')

print('Model 7 - Ridge Linear Regressor')
print('Coefficient of determination - ', model7.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model7.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model7.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model7.predict(X_test)))
print('Std of predictions - ', np.std(model7.predict(X_test)-y_test))
print('')

print('Model 8 - RidgeCV Linear Regressor')
print('Coefficient of determination - ', model8.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model8.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model8.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model8.predict(X_test)))
print('Std of predictions - ', np.std(model8.predict(X_test)-y_test))
print('')

print('Model 9 - ElasticNet Linear Regressor')
print('Coefficient of determination - ', model9.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model9.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model9.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model9.predict(X_test)))
print('Std of predictions - ', np.std(model9.predict(X_test)-y_test))
print('')

print('Model 10 - ElasticNetCV Linear Regressor')
print('Coefficient of determination - ', model10.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model10.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model10.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model10.predict(X_test)))
print('Std of predictions - ', np.std(model10.predict(X_test)-y_test))
print('')

print('Model 11 - Lasso Linear Regressor')
print('Coefficient of determination - ', model11.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model11.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model11.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model11.predict(X_test)))
print('Std of predictions - ', np.std(model11.predict(X_test)-y_test))
print('')

print('Model 12 - LassoCV Linear Regressor')
print('Coefficient of determination - ', model12.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model12.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model12.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model12.predict(X_test)))
print('Std of predictions - ', np.std(model12.predict(X_test)-y_test))
print('')

print('Model 13 - Voting Regressor')
print('Coefficient of determination - ', model13.score(X_test, y_test))
print('Pearson R - ', pearsonr(y_test,model13.predict(X_test)))
print('Mean absolute error - ', mean_absolute_error(y_test,model13.predict(X_test)))
print('Root mean square error - ', mean_squared_error(y_test, model13.predict(X_test)))
print('Std of predictions - ', np.std(model13.predict(X_test)-y_test))
print('')

Model 1 - Gradient Boosting Regressor
Coefficient of determination -  0.6428212403389172
Pearson R -  PearsonRResult(statistic=0.8165293257293859, pvalue=9.14994714992711e-28)
Mean absolute error -  6.268488114623359
Root mean square error -  73.12693726466588
Std of predictions -  8.539285779234369

Model 3 - Random Forest Regressor
Coefficient of determination -  0.650620959129258
Pearson R -  PearsonRResult(statistic=0.8131866381403478, pvalue=2.2228379573407677e-27)
Mean absolute error -  5.871171171171172
Root mean square error -  71.53006306306305
Std of predictions -  8.420525586235442

Model 4 - AdaBoost Forest Regressor
Coefficient of determination -  0.5516118891650579
Pearson R -  PearsonRResult(statistic=0.8045995752944524, pvalue=2.0078956163416556e-26)
Mean absolute error -  7.559338510879893
Root mean square error -  91.800669452914
Std of predictions -  9.422837211719665

Model 5 - Decision Tree Regressor
Coefficient of determination -  0.4694961978769001
Pearson R -  P

In [227]:
#investigate models
#print(pd.DataFrame(zip(X.columns, model2.coef_)).to_string()) # check coefficients of linearregressor
# linear regressor seems to require scaling to size of output e.g. 0-100

In [228]:
# quick plot of test results
test_labels = pd.merge(y_test, data['Game'], left_index= True, right_index = True) #join the game name back in to rating scores
test_scores = pd.DataFrame({'Game': test_labels['Game'], 'Predicted Rating':model8.predict(X_test), 'Actual Rating': test_labels['Rating']}) #join the test set to the predictions
fig = px.scatter(test_scores, x="Actual Rating", y="Predicted Rating", hover_name= 'Game')
fig

In [229]:
# save the best model
joblib.dump(model8, 'gbrmodel.joblib')
#save the data
data.to_pickle('../data/processed_data.pkl')
X.to_pickle('../data/X.pkl')
y.to_pickle('../data/y.pkl')
X_test.to_pickle('../data/X_test.pkl')
y_test.to_pickle('../data/y_test.pkl')